# Final Project

## 파이썬 환경 설정 및 확인

In [ ]:
# C:/Users/TECH2_22/이도원/usingcloud_final/venv/Scripts/python.exe

In [ ]:
import os

os.getcwd()

In [ ]:
print('hello world')

In [ ]:
!python --version

## 데이터 로드 및 설정

### 최초 1회만 실행

In [1]:
import pandas as pd
import numpy as np
import json

with open('./data/train_original.json', encoding='utf-8') as train_f:
    train_data = json.loads(train_f.read())
    train_df = pd.DataFrame(train_data['documents'])

with open('./data/valid_original.json', encoding='utf-8') as valid_f:
    valid_data = json.loads(valid_f.read())
    valid_df = pd.DataFrame(valid_data['documents'])

In [2]:
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)

In [3]:
train_df['body'] = train_df.text.apply(lambda x : sentence_validation(extract_body(x)))
train_df.head(2)

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive,body
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...


## eda

In [ ]:
train_df.head(3)

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df.head(1).T

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.columns

## 전처리

In [ ]:
def extract_body(article) -> str:
    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence: 
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)

In [6]:
from tokenization_kobert import KoBertTokenizer
import sentencepiece as spm

# sentencepiece 라이브러리 설치 필요
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [6]:
# 길이가 가장 긴 텍스트 탐색
# index : 66068
# 길이 : 13910
new_df.text.str.len().sort_values(ascending=False)

66068     13910
36893      1963
182051     1959
187153     1955
105903     1954
          ...  
65237         0
157929        0
157927        0
66            0
156078        0
Name: text, Length: 243983, dtype: int64

In [27]:
SEQ_LEN = 7500

len(tokenizer.encode(new_df['text'][66068], max_length=SEQ_LEN, padding='max_length'))

7500

In [29]:
result = tokenizer.encode_plus(new_df['text'][0], max_length=SEQ_LEN, padding="max_length")
result

{'input_ids': [2, 4012, 6003, 5666, 5859, 5330, 517, 6756, 517, 5468, 7144, 6236, 6116, 1226, 7203, 4999, 7789, 3567, 3451, 3004, 7794, 517, 6756, 2714, 7259, 7236, 3997, 4555, 7574, 5019, 54, 517, 6756, 2714, 7259, 7234, 5760, 517, 6353, 6116, 3060, 6889, 1498, 6896, 517, 6353, 1655, 2573, 6093, 7170, 6241, 7098, 517, 7546, 1815, 1567, 3930, 6242, 3060, 7083, 517, 6353, 6984, 3820, 2828, 7389, 6116, 2355, 7207, 7857, 4137, 5782, 54, 3450, 4018, 7095, 1498, 1567, 3930, 6241, 3969, 6312, 990, 6198, 7202, 7086, 4014, 611, 6150, 402, 7095, 3211, 553, 94, 7119, 529, 6150, 617, 247, 402, 6079, 46, 2809, 6705, 7881, 7318, 7491, 7088, 5134, 46, 2959, 5512, 6896, 4746, 7869, 54, 4331, 6503, 1651, 7086, 578, 5712, 6516, 517, 6756, 2341, 7342, 6424, 5557, 2232, 1507, 7318, 6896, 533, 367, 18, 145, 7724, 40, 3704, 517, 6353, 3647, 6995, 1567, 3930, 6242, 3969, 6312, 7828, 1508, 7119, 7100, 54, 4331, 1651, 3930, 6241, 7086, 529, 5718, 7088, 4868, 1562, 5718, 7095, 2048, 3930, 6241, 7096, 5000, 588

In [ ]:
from tqdm import tqdm
from tokenization_kobert import KoBertTokenizer

import re
import numpy as np
import sentencepiece as spm

# sentencepiece 라이브러리 설치 필요
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

SEQ_LEN = 7500

token_ids = []
token_masks = []
token_seqments = []

train_labels = []

for idx in tqdm(range(len(new_df.text))):
    train_sentence = new_df.text.iloc[idx]

    cleaned_sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", train_sentence) # [ whitespaces, 숫자, 영문 알파벳, 한글(+자모음) ]이 아닌 것을 공백으로 치환 (특수문자 제거)

    encoded_dict = tokenizer.encode_plus(
        text = cleaned_sentence,
        padding = 'max_length',
        truncation = True,
        max_length = SEQ_LEN,
    )

    token_ids.append(encoded_dict['input_ids'])
    token_masks.append(encoded_dict['token_type_ids'])
    token_seqments.append(encoded_dict['attention_mask'])

train_inputs = (np.array(token_ids), np.array(token_masks), np.array(token_seqments))
train_inputs

In [52]:
np.array(token_ids)

MemoryError: Unable to allocate 6.82 GiB for an array with shape (243983, 7500) and data type int32

pipeline에서 선택가능한 종류

- feature-extraction (텍스트에 대한 벡터 표현 제공)
- fill-mask
- ner (named entity recognition, 개체명 인식)
- question-answering
- sentiment-analysis
- summarization
- text-generation
- translation
- zero-shot-classification

In [16]:
from transformers import pipeline

classifier = pipeline("summarization")
classifier(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
    """,
    max_length = 1000
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 1000, but you input_length is only 289. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=144)


[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil, electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India, as well as other industrial countries in Europe and Asia, continue to encourage and advance engineering .'}]

### 여기서부터 KoBART

In [7]:
import tokenizers

tokenizers.__version__

'0.10.3'

InvalidVersion: Invalid version: '0.10.1,<0.11' 오류 발생 시 대처법

1. `transformers` 패키지를 최신버전으로 업그레이드 한다.
2. `is_tokenizers_available`함수의 위치를 파악한다. (import_utils.py)
3. `is_tokenizers_available`함수를 import 한 파일에서 원 코드를 주석처리하고, utils.import_utils 에서 import 하게 끔 코드를 새로 작성한다.<br>
3.1 `from utils.import_utils import is_tokenizers_available`
4. vscode를 재시작한다.

In [5]:
from kobart import get_kobart_tokenizer

kobart_tokenizer = get_kobart_tokenizer()
kobart_tokenizer.tokenize("안녕하세요. 한국어 BART 입니다.🤣:)l^o")

c:\Users\TECH2_22\이도원\usingcloud_final\.cache\kobart_base_tokenizer_cased_cf74400bce.zip[██████████████████████████████████████████████████]


['▁안녕하', '세요.', '▁한국어', '▁B', 'A', 'R', 'T', '▁입', '니다.', '🤣', ':)', 'l^o']

In [7]:
from transformers import BartModel
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer

kobart_tokenizer = get_kobart_tokenizer()
model = BartModel.from_pretrained(get_pytorch_kobart_model())
inputs = kobart_tokenizer(
    ["""
    질병관리청은 23일 지방자치단체가 보건당국과 협의 없이 단독으로 인플루엔자(독감) 백신 접종 중단을 결정해서는 안 된다는 입장을 밝혔다.
    질병청은 이날 참고자료를 배포하고 “향후 전체 국가 예방접종사업이 차질 없이 진행되도록 지자체가 자체적으로 접종 유보 여부를 결정하지 않도록 안내를 했다”고 설명했다.
    독감백신을 접종한 후 고령층을 중심으로 전국에서 사망자가 잇따르자 서울 영등포구보건소는 전날, 경북 포항시는 이날 관내 의료기관에 접종을 보류해달라는 공문을 내려보냈다. 이는 예방접종과 사망 간 직접적 연관성이 낮아 접종을 중단할 상황은 아니라는 질병청의 판단과는 다른 것이다.
    질병청은 지난 21일 전문가 등이 참여한 ‘예방접종 피해조사반’의 분석 결과를 바탕으로 독감 예방접종 사업을 일정대로 진행하기로 했다. 특히 고령 어르신과 어린이, 임신부 등 독감 고위험군은 백신을 접종하지 않았을 때 합병증 피해가 클 수 있다면서 접종을 독려했다. 하지만 접종사업 유지 발표 이후에도 사망 보고가 잇따르자 질병청은 이날 ‘예방접종 피해조사반 회의’와 ‘예방접종 전문위원회’를 개최해 독감백신과 사망 간 관련성, 접종사업 유지 여부 등에 대해 다시 결론 내리기로 했다. 회의 결과는 이날 오후 7시 넘어 발표될 예정이다.
    """],
    return_tensors='pt'
)
model(inputs['input_ids'])

using cached model. c:\Users\TECH2_22\이도원\usingcloud_final\.cache\kobart_base_tokenizer_cased_cf74400bce.zip


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


using cached model. c:\Users\TECH2_22\이도원\usingcloud_final\.cache\kobart_base_cased_ff4bda5738.zip


Seq2SeqModelOutput(last_hidden_state=tensor([[[ 0.9474,  2.3639, -0.2549,  ...,  0.2183,  3.8118,  1.1812],
         [-1.2098,  4.5790, -2.8945,  ...,  1.7517,  2.2744, -3.6560],
         [-0.1812,  5.0438, -0.7288,  ...,  4.5124,  4.6417, -0.8159],
         ...,
         [-2.7491,  5.8629, -2.1584,  ...,  5.3252, -1.7025, -0.7520],
         [-0.9481,  6.7857, -0.5848,  ...,  4.6638,  0.6773, -1.9789],
         [-0.8366,  6.4286, -1.3465,  ...,  4.3974,  0.6303, -1.0256]]],
       grad_fn=<NativeLayerNormBackward>), past_key_values=((tensor([[[[-9.7980e-02, -6.6584e-01, -1.8089e+00,  ...,  9.6023e-01,
           -1.8818e-01, -1.3252e+00],
          [ 1.9448e-01, -1.1043e+00, -8.7914e-01,  ...,  1.0640e+00,
           -1.1785e-01, -5.8921e-01],
          [ 4.5271e-04, -7.2249e-02,  8.6127e-01,  ..., -2.3345e-01,
            5.6567e-01, -1.3224e+00],
          ...,
          [-1.3626e-01, -1.9508e-01,  1.8143e+00,  ...,  8.2038e-01,
           -2.8528e-01,  7.5096e-01],
          [ 2.626

In [55]:
import sys

local_vars = list(locals().items())

for var, obj in local_vars:
    obj_size = sys.getsizeof(obj) # Return the size of object in bytes -> 1064 * 1064 bytes == 1 MB
    if (obj_size > 1064 * 1064) and (not var.startswith('_')): 
        print(var, int(obj_size / (1064 * 1064)), 'MB')

sys.getwindowsversion(major=10, minor=0, build=19044, platform=2, service_pack='')

## 여기가 끝